In [4]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = None
pd.options.display.max_rows = None

pd.options.display.max_colwidth=-1

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim import corpora, models
import gensim
import nltk
nltk.download("stopwords")


%autoreload 2

# Visualizations
import plotly.express as px

import matplotlib.pyplot as plt
import seaborn as sns
import re

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim

[nltk_data] Downloading package stopwords to C:\Users\Abhishek
[nltk_data]     Baranwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

UsageError: Line magic function `%autoreload` not found.


In [6]:
df=pd.read_csv("amazon_review.csv" ,encoding='utf-8')

In [7]:
df.head()

Unnamed: 0  \
0  0            
1  1            
2  2            
3  3            
4  4            

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10422 entries, 0 to 10421
Data columns (total 2 columns):
Unnamed: 0    10422 non-null int64
0             10422 non-null object
dtypes: int64(1), object(1)
memory usage: 162.9+ KB


In [16]:
df['0'][0]

"I first owned a Bose QC35, however, I felt Bose was not innovating in terms of sound control and technology. This led me to buy the Sony WH-1000XM2 which I used for over a year without any trouble. I was completely happy with them. When Sony released Sony WH-1000XM3, I took the plunge and bought them in November 2018. After a month of using them, I began to hear crackling noises from the ear cups. I ignored them thinking they were some issue with the Bluetooth connection. This issue pursued and the noises became so unbearable that when they randomly came up, I had to immediately remove the headset from my wearing. At this point, I decided to Google the issue and was shocked to see that this was a common issue in most Sony WH-1000XM3.Most of the above issues came up in the warranty period and most, if not everyone had their headsets replaced, however, the shocking part is that many complain that the issues persisted in the new headsets too. Looking at the bright side and the brand name

# Tokenization

In [17]:
tokenizer = RegexpTokenizer(r'\w+')

In [23]:
doc_1 = df['0'][0]

In [24]:
tokens = tokenizer.tokenize(doc_1.lower())

In [25]:
print('{} characters in string vs {} words in a list'.format(len(doc_1),                                                             len(tokens)))
print(tokens[:10])

7522 characters in string vs 1443 words in a list
['i', 'first', 'owned', 'a', 'bose', 'qc35', 'however', 'i', 'felt', 'bose']


# Stop Words

In [26]:
nltk_stpwd = stopwords.words('english')

In [27]:
print(len(set(nltk_stpwd)))
print(nltk_stpwd[:10])

179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [28]:
stopped_tokens = [token for token in tokens if not token in nltk_stpwd]
print(stopped_tokens[:10])

['first', 'owned', 'bose', 'qc35', 'however', 'felt', 'bose', 'innovating', 'terms', 'sound']


# Stemming

In [29]:
sb_stemmer = SnowballStemmer('english')

In [30]:
stemmed_tokens = [sb_stemmer.stem(token) for token in stopped_tokens]
print(stemmed_tokens)

['first', 'own', 'bose', 'qc35', 'howev', 'felt', 'bose', 'innov', 'term', 'sound', 'control', 'technolog', 'led', 'buy', 'soni', 'wh', '1000xm2', 'use', 'year', 'without', 'troubl', 'complet', 'happi', 'soni', 'releas', 'soni', 'wh', '1000xm3', 'took', 'plung', 'bought', 'novemb', '2018', 'month', 'use', 'began', 'hear', 'crackl', 'nois', 'ear', 'cup', 'ignor', 'think', 'issu', 'bluetooth', 'connect', 'issu', 'pursu', 'nois', 'becam', 'unbear', 'random', 'came', 'immedi', 'remov', 'headset', 'wear', 'point', 'decid', 'googl', 'issu', 'shock', 'see', 'common', 'issu', 'soni', 'wh', '1000xm3', 'issu', 'came', 'warranti', 'period', 'everyon', 'headset', 'replac', 'howev', 'shock', 'part', 'mani', 'complain', 'issu', 'persist', 'new', 'headset', 'look', 'bright', 'side', 'brand', 'name', 'confid', 'new', 'firmwar', 'releas', 'would', 'fix', 'everyth', 'thought', 'soon', 'releas', 'although', 'issu', 'list', 'hope', 'new', 'firmwar', 'releas', 'fix', 'issu', 'still', 'could', 'get', 'heads

In [37]:
num_reviews = df.shape[0]
doc_set = [df['0'][i] for i in range(num_reviews)]

In [38]:
texts = []

In [39]:
for doc in doc_set:
    tokens = tokenizer.tokenize(doc.lower())
    stopped_tokens = [token for token in tokens if not token in nltk_stpwd]
    stemmed_tokens = [sb_stemmer.stem(token) for token in stopped_tokens]
    texts.append(stemmed_tokens)
    
print(texts[1])

['first', 'nois', 'cancel', 'headphon', 'expert', 'opinion', 'p', 'nois', 'cancel', 'like', 'use', 'tight', 'ear', 'headphon', 'comfort', 'block', 'extern', 'nois', 'like', 'ac', 'fan', 'rain', 'etc', 'block', 'mean', '100', '80', '90', 'play', 'music', 'hear', 'anyth', 'outsid', 'world', 'said', 'ear', 'headphon', 'slight', 'pain', 'tight', 'ear', 'headphon', 'replac', 'built', 'nois', 'cancel', 'point', 'peopl', 'never', 'experienc', 'nois', 'cancel', 'headphon', 'life', 'nois', 'cancel', 'anoth', 'level', 'believ', 'actual', 'get', 'chanc', 'use', 'youtub', 'review', 'give', 'real', 'experi', 'nois', 'cancel', 'headphon', 'use', 'watch', 'youtub', 'video', 'tutori', 'write', 'code', 'brainstorm', 'travel', 'go', 'bed', 'hear', 'song', 'sleep', 'need', 'disconnect', 'annoy', 'world', 'around', 'batteri', 'life', 'realli', 'good', 'usb', 'c', 'mean', 'one', 'less', 'cabl', 'fast', 'charg', 'good', 'touch', 'control', 'awesom', 'put', 'right', 'hand', 'right', 'ear', 'cup', 'come', 'ba

# Create a dictionary using corpora

In [40]:
texts_dict = corpora.Dictionary(texts)
texts_dict.save('elec_review.dict') 
print(texts_dict)

Dictionary(7116 unique tokens: ['000', '1', '10', '100', '1000xm2']...)


In [41]:
import operator
print("IDs 1 through 10: {}".format(sorted(texts_dict.token2id.items(), key=operator.itemgetter(1), reverse = False)[:10]))

IDs 1 through 10: [('000', 0), ('1', 1), ('10', 2), ('100', 3), ('1000xm2', 4), ('1000xm3', 5), ('12th', 6), ('14th', 7), ('15th', 8), ('16th', 9)]


In [43]:
print(df['0'].str.contains("complaint").value_counts()) 
print(df['0'].str.contains("lot").value_counts())

False    10328
True     94   
Name: 0, dtype: int64
False    9814
True     608 
Name: 0, dtype: int64


In [44]:
texts_dict.filter_extremes(no_below=30, no_above=0.15) # inlace filter
print(texts_dict)
print("top terms:")
print(sorted(texts_dict.token2id.items(), key=operator.itemgetter(1), reverse = False)[:10])

Dictionary(1277 unique tokens: ['000', '1', '10', '100', '1000xm2']...)
top terms:
[('000', 0), ('1', 1), ('10', 2), ('100', 3), ('1000xm2', 4), ('1000xm3', 5), ('18', 6), ('2', 7), ('2018', 8), ('2019', 9)]


# Creating Bag of Words

In [45]:
corpus = [texts_dict.doc2bow(text) for text in texts]
len(corpus)

10422

In [46]:
%%time 
gensim.corpora.MmCorpus.serialize('oneplusreviews.mm', corpus)

Wall time: 707 ms


# Training an LDA model

In [47]:
lda_model = gensim.models.LdaModel(corpus,alpha='auto', num_topics=5,id2word=texts_dict, passes=20)#Choosing the number of topics based on various categories of electronics on Amazon
lda_model.show_topics(num_topics=5,num_words=5)
raw_query = 'one plus mobiles'
query_words = raw_query.split()
query = []
for word in query_words:
    # ad-hoc reuse steps from above
    q_tokens = tokenizer.tokenize(word.lower())
    q_stopped_tokens = [word for word in q_tokens if not word in nltk_stpwd]
    q_stemmed_tokens = [sb_stemmer.stem(word) for word in q_stopped_tokens]
    query.append(q_stemmed_tokens[0])
    
print(query)# Words in query will be converted to ids and frequencies  
id2word = gensim.corpora.Dictionary()
_ = id2word.merge_with(texts_dict) # garbage# Convert this document into (word, frequency) pairs
query = id2word.doc2bow(query)
print(query)#Create a sorted list
sorted_list = list(sorted(lda_model[query], key=lambda x: x[1]))
sorted_list#Assessing least related topics
lda_model.print_topic(sorted_list[0][0]) #least related#Assessing most related topics
lda_model.print_topic(sorted_list[-1][0])#most related

[(0, '0.018*"5" + 0.018*"connect" + 0.017*"2" + 0.017*"charg" + 0.016*"call"'),
 (1,
  '0.012*"app" + 0.011*"hour" + 0.010*"time" + 0.010*"would" + 0.009*"listen"'),
 (2,
  '0.011*"hear" + 0.010*"audio" + 0.010*"clear" + 0.009*"amaz" + 0.008*"10"'),
 (3,
  '0.020*"amazon" + 0.015*"buy" + 0.014*"month" + 0.013*"day" + 0.013*"return"'),
 (4,
  '0.030*"earphon" + 0.016*"also" + 0.014*"awesom" + 0.013*"comfort" + 0.012*"high"')]

['one', 'plus', 'mobil']
[(298, 1), (778, 1)]


[(2, 0.02818896),
 (0, 0.04260422),
 (3, 0.052571602),
 (4, 0.09745398),
 (1, 0.77918124)]

'0.011*"hear" + 0.010*"audio" + 0.010*"clear" + 0.009*"amaz" + 0.008*"10" + 0.008*"m50x" + 0.008*"bose" + 0.008*"wire" + 0.007*"review" + 0.007*"headset"'

'0.012*"app" + 0.011*"hour" + 0.010*"time" + 0.010*"would" + 0.009*"listen" + 0.009*"volum" + 0.009*"much" + 0.008*"featur" + 0.008*"connect" + 0.008*"control"'

# Above are the top 5 words associated with 1 topic. The float next to each word is the weight showing how much the given word influences this specific topic. We can interpret that here the topic might be close to sony headphones category